In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import BertTokenizer

from typing import *

from MEOW_Models import QA_models, MT_models
from MEOW_Utils import QA_utils, MT_utils

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 4
EPOCH_NUM = 10
INPUT_FILE_PATH = r'C:\Users\Administrator\codeblocks_workspace\MEOW\train-v2.0.json'
PRETRAINED_MODULE_NAME = 'bert-base-uncased'

In [5]:
#處理好 dataframe
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODULE_NAME)
df_train = MT_utils.get_SQuAD_df(INPUT_FILE_PATH, tokenizer)

Reading the json file
processing...


c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:100: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  js = pd.io.json.json_normalize(file , record_path )
c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:101: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  m = pd.io.json.json_normalize(file, record_path[:-1])
c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:102: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  r = pd.io.json.json_normalize(file,record_path[:-2])
c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:109: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()


shape of the dataframe is (130319, 6)
Done


c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['answer_start'] = df_train['answer_start'].apply(lambda x : -1 if np.isnan(x) else int(x))
c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'] = df_train['text'].apply(lambda x : x if type(x) is str else '')
c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\MT_utils.py:140: Setting

In [3]:
#處理好dataset dataloader model等等

bert_model = MT_models.BertWithoutEmbedding.from_pretrained(PRETRAINED_MODULE_NAME)
GETEMBEDDING_helper =  MT_utils.get_bert_element(bertmodel=bert_model)
embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

train_dataset = QA_utils.QAdataset(df_train, tokenizer=tokenizer)
train_loader = QA_utils.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=QA_utils.collate_batch)

QA_model = QA_models.Bert_QA(model=bert_model, embedding_layer=embedding_layer, device=DEVICE)
QA_model.to(DEVICE)

EPOCH_NUM = 10
optimizer = torch.optim.SGD(QA_model.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.Adam(QA_model.parameters(), lr=0.001, betas=(0.88, 0.95), eps=1e-08)

H = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithoutEmbedding: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertWithoutEmbedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithoutEmbedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 訓練

for epoch in range(EPOCH_NUM):
    print("the {:d} iter :".format(epoch+1))

    QA_model.train()
    training_loss = 0.0
    training_correct = 0.0

    for data in train_loader:
        input_ids, mask, token, SEPind, Start_pos, End_pos = data
        
        # input_ids = input_ids.type(torch.IntTensor)
        # mask = mask.type(torch.IntTensor)
        # labels = labels.type(torch.LongTensor)


        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        
        loss = QA_model(input_ids, mask, token, SEPind, Start_pos, End_pos)
        
        # print(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # correct_num = (Start_pred == Start_1hot).type(torch.int).sum()
        
        training_loss += loss.item()

        # training_correct += correct
        

    avg_loss = training_loss / len(train_loader)

    H['train_loss'].append(avg_loss)
    # H['train_acc'].append(avg_correct)
    print("Train loss: {:.6f}".format(avg_loss))
#

the 1 iter :


KeyboardInterrupt: 

In [ ]:
# 繪製結果

import matplotlib
import matplotlib.pyplot as plt

# tensor to float
H['train_loss'] = [float(i) for i in H['train_loss']]
# H['train_acc'] = [float(i) for i in H['train_acc']]
# H['val_loss'] = [float(i) for i in H['val_loss']]
# H['val_acc'] = [float(i) for i in H['val_acc']]

# loss
plt.figure()
plt.title("Loss")
plt.xlabel("EPOCH")
plt.ylabel("Loss")
plt.plot(H["train_loss"], label="test_loss")
# plt.plot(H["val_loss"], label="test_loss")
plt.xticks(np.arange(EPOCH_NUM), range(1,EPOCH_NUM+1,1))
plt.show()

# accuracy
# plt.figure()
# plt.title("Test Accuracy")
# plt.xlabel("EPOCH")
# plt.ylabel("Accuracy")
# plt.plot(H["train_acc"], label="test_acc")
# # plt.plot(H["val_acc"], label="test_acc")
# plt.xticks(np.arange(6), range(1,7,1))
# plt.show()

